# Variational Quantum Classifier over 3 Datasets via Angle Encoding

### Dataset Preprocessing

In [13]:
from sklearn import preprocessing
from pennylane import numpy as np
import pandas as pd, random as rand
                 
def preprocess(df):
    # Label Encode if target is a string
    if isinstance(df.target.unique().sum(), str):
        label_encoder = preprocessing.LabelEncoder()
        df.target = label_encoder.fit_transform(df.target)
    
    # Truncate to binary classification
    df = df[(df.target == 0) | (df.target == 1)]
    
    # Balance the data if necessary via undersampling.
    if df.target.value_counts()[0] != df.target.value_counts()[1]:
        df = [df[df.target == 0], df[df.target == 1]]
        df = [dataframe.sample(n=50, random_state=rand.randint(0,100)) for dataframe in df]
        df = pd.concat(df)
    
    # Shuffle the dataframe
    df = df.sample(frac=1, random_state=rand.randint(0,100)).to_numpy()
             
    return df
    


In [15]:
paths = ['~/Documents/QML/iris.data', '~/Documents/QML/banknote.data','~/Documents/QML/transfusion.data']

raw_dfs = [pd.read_csv(dataset_path, names=['a0','a1','a2','a3', 'target']) for dataset_path in paths]
dfs = [preprocess(df) for df in raw_dfs]

X = dfs[0][:, :-1]
Y = dfs[0][:, -1] * 2 + -1

rot_features = 2 * np.pi * (X - np.min(X)) / (np.max(X) - np.min(X))

rot_train = rot_features[index[:num_train]]
Y_val = Y[index[num_train:]]
Y_train = Y[index[:num_train]]
rot_val = rot_features[index[num_train:]]


### Machine Learning Setup

In [6]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):
    num_correct = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            num_correct += 1
    return num_correct / len(labels)

In [7]:
from pennylane.optimize import AdamOptimizer


num_qubits = 4
num_layers = 3

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = qml.optimize.AdamOptimizer(stepsize=0.1)
batch_size = 5

### The Quantum Circuit
https://docs.pennylane.ai/en/stable/code/api/pennylane.AngleEmbedding.html

In [8]:
from pennylane.optimize import AdamOptimizer
import pennylane as qml

dev = qml.device('default.qubit', wires=NUM_WIRES)

def rotation_encoding(features, evolution, entangle=False):
    assert(issubclass(evolution, qml.operation.Operation))
    evolution(features[0], wires=0)
    evolution(features[1], wires=1)
    evolution(features[2], wires=2)
    evolution(features[3], wires=3)
    if entangle:
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 0])

def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

@qml.qnode(dev)
def circuit(weights, feature_vector):
    rotation_encoding(feature_vector, qml.RX, True)
    
    for w in weights:
        layer(w)

    return qml.expval(qml.PauliY(0))

def variational_classifier(weights, bias, angles):
    return circuit(weights, angles) + bias

def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)

### Execution

In [17]:
weights = weights_init
bias = bias_init
#print(qml.draw(_circuit)(weights, feats[0], embed, arg))

for it in range(100): #changed from 60
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = rot_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in rot_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in rot_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    temp_cost = cost(weights, bias, rot_train, Y)
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, temp_cost, acc_train, acc_val)
    )
    if acc_val == 1:
        break

Iter:     1 | Cost: 0.8716049 | Acc train: 0.1733333 | Acc validation: 0.0800000 
Iter:     2 | Cost: 0.8652887 | Acc train: 0.1466667 | Acc validation: 0.0800000 
Iter:     3 | Cost: 0.8584291 | Acc train: 0.1333333 | Acc validation: 0.0800000 
Iter:     4 | Cost: 0.8519325 | Acc train: 0.1333333 | Acc validation: 0.0800000 
Iter:     5 | Cost: 0.8442251 | Acc train: 0.1333333 | Acc validation: 0.0800000 
Iter:     6 | Cost: 0.8360013 | Acc train: 0.1466667 | Acc validation: 0.1200000 
Iter:     7 | Cost: 0.8270835 | Acc train: 0.2000000 | Acc validation: 0.2000000 
Iter:     8 | Cost: 0.8175643 | Acc train: 0.2533333 | Acc validation: 0.2000000 
Iter:     9 | Cost: 0.8079747 | Acc train: 0.2933333 | Acc validation: 0.2000000 
Iter:    10 | Cost: 0.7989345 | Acc train: 0.3600000 | Acc validation: 0.2800000 
Iter:    11 | Cost: 0.7909905 | Acc train: 0.3866667 | Acc validation: 0.2800000 
Iter:    12 | Cost: 0.7842212 | Acc train: 0.4400000 | Acc validation: 0.3200000 
Iter:    13 | Co

KeyboardInterrupt: 